## Project 1: Resume Parser

### A simple example of a commercial use case: parsing and summarizing a resume

We will Gradio to quickly prototype this use of the OpenAI API

In [ ]:
# imports

import os
from dotenv import load_dotenv
import openai
import json
from openai import OpenAI
import gradio as gr

In [ ]:
# constants

MODEL = "gpt-4o"

In [ ]:
# Load environment variables in a file called .env

load_dotenv()

In [ ]:
# Load sample HR-JSON file

with open('sample.json', 'r') as f:
    sample = f.read()

In [ ]:
# Also load my resume as an example

with open('my_resume.txt', 'r') as f:
    me = f.read()

In [ ]:
# Create the OpenAI client

client = OpenAI()

In [ ]:
# Create the prompt, comprising of system instructions, and a specific request

instructions = f"""You are an assistant that can convert a resume in plain text precisely into the industry standard HR-JSON format.
Here is an example of HR-JSON populated for a candidate.
HR-JSON elements are optional, so if a section isn't populated in a resume, then it should be omitted in the HR-JSON.
Example HR-JSON:\n{sample}"""

request = "Please convert this resume to HR-JSON:\n"

In [ ]:
# Let's try it for my resume

result = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": request + me}
          ],
        response_format={ "type": "json_object" })
print(result.choices[0].message.content)

In [ ]:
# Wrap this in a method

def parse(text_resume):
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": request + text_resume}
          ],
        response_format={ "type": "json_object" },
        stream=True
        )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# Bringing up a view in Gradio is incredibly easy!

view = gr.Interface(
    fn=parse,
    inputs=[gr.Textbox(label="Text Resume", lines=10)],
    outputs=[gr.Textbox(label="HR-JSON Resume", lines=10)],
    allow_flagging="never"
)

view.launch()

In [ ]:
# A simple variation to generate a summary of this resume, a very common Gen AI use case

instructions = """You are an assistant that can create a short summary of a candidate based on their resume.
The summary should be 4-5 sentences long and capture their experience and education."""

request = "Please summarize this candidate based on their resume:\n"

In [ ]:
# Generate as before

def parse(text_resume):
    response = client.chat.completions.create(
          model=MODEL,
          messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": request + text_resume}
          ],
        stream=True
        )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# And in Gradio again

view = gr.Interface(
    fn=parse,
    inputs=[gr.Textbox(label="Text Resume", lines=10)],
    outputs=[gr.Textbox(label="Summary", lines=10)],
    allow_flagging="never"
)

view.launch()